###### 0

<div style="background-image: linear-gradient(145deg, rgba(35, 47, 62, 1) 0%, rgba(0, 49, 129, 1) 40%, rgba(32, 116, 213, 1) 60%, rgba(244, 110, 197, 1) 85%, rgba(255, 173, 151, 1) 100%); padding: 1rem 2rem; width: 95%"><img style="width: 60%;" src="images/MLU_logo.png"></div>

# MLU Operationalizing Generative AI with LLMOps 
# <a name="p0">Lab 2: Enhancing the capabilities of the RAG system</a>

In this lab you will make improvements to the deployed application to enhance its capabilities. More concretely, you will first make changes to the database of documents that powers the RAG application and that is stored in Amazon Kendra. You will practice how to add or replace information to keep the knowledge base up to date and relevant.

The second improvement is related to the response returned by the system. When using RAG to enhance the answer from the LLM, it is useful to provide links to the retrieved documents that are inserted into the context. You will learn how to modify the function that handles the system response to include this information in its output. This feature can support the addition of references that can be useful for the user. In addition to making modifications to the main app component that serves the response, you will practice how to adapt the tests to be able to build the modified app. 

## Table of Contents
1. [Extend the knowledge database that powers RAG](#1)
2. [Add links to relevant documents in the API response](#2)
3. [Commit your changes and update the pipeline](#3)

<br/>
<div style="display: flex; align-items: center; justify-content: left; background-color:#330066; width:99%;"> 
        <img style="float: left; max-width: 100%; max-height:100%; margin: 15px;" src="images/MLU_robot.png" alt="MLU robot" width="100" height="100"/>
    <span style="color: white; padding-left: 10px; align: left; margin: 15px;">
        This notebook assumes that you have already completed the following tasks:<br/>
        <ul>
            <li>Ran Lab 1: Interacting with the LLM-powered application.</li>
            <li>Deployed changes to the system so that it returns a properly formatted response within <code style="color: lightcoral;">&lt;answer&gt;</code> tags even for questions not related to AWS Lambda.</li>
        </ul>
    </span>
</div>

### Select the Jupyter Kernel

In order to run this notebook, you need to use a Kernel. We will use the Kernel from the Python virtual environment provided with this package. 

**In VSCode**
  - Click on "Select Kernel" on the top right of this window.
  - Click on "Python Environments" on the text input bar at the top of this window.
  - Select the `.venv (Python 3.12.x)` Virtual env, located in `{WorkspaceRoot}/src/{Alias}MLUCourseLLMOpsExperiment/.venv/bin/python`
  - Double check that the Kernel shown on the top right of this window reads `.venv (Python 3.12.x)`

  

### A note about your development environment

One of the key goals of this lab is to give you an experience as similar as possible to a real LLM operationalization process. 

Besides the data science environment based on Jupyter notebooks, where you can experiment with LLM models, you will also be required to run commands in a terminal window.

Every time you see  an instruction to open a terminal and run a command, feel free to choose the terminal you are more comfortable with: 
+ A Terminal window with SSH to your Cloud Desktop, or
+ The VS Code Terminal window. In this case, make sure to open the terminal in the same window that is connected to your Cloud Destop environment.

***

We will reuse code from the previous lab to recreate the function that makes live requests to the system.

In [2]:
# Install and import the library that support AWS SigV4 requests
!pip3 install -q requests-auth-aws-sigv4

from IPython.display import Markdown, display

import boto3
import json
import requests
from requests_auth_aws_sigv4 import AWSSigV4

from awscurl.awscurl import make_request

session = boto3.Session(profile_name="MLU-LLMOps-Burner")

alias = %env USER
MAIN_PACKAGE_NAME = f"{alias.capitalize()}MLUCourseLLMOps"

aws_auth = AWSSigV4("cloudformation", region="us-west-2", session=session)
url = f"https://cloudformation.us-west-2.amazonaws.com?Action=DescribeStacks&StackName={MAIN_PACKAGE_NAME}-Service-alpha"
headers = {"Accept": "application/json"}

r = requests.request("GET", url, auth=aws_auth, headers=headers)
outputs = r.json()["DescribeStacksResponse"]["DescribeStacksResult"]["Stacks"][0]["Outputs"]
api_endpoint = [output for output in outputs if output["ExportName"]==f"{MAIN_PACKAGE_NAME}-ApiUrl"][0]["OutputValue"]

def make_live_request(question):
    credentials = session.get_credentials() 
    response = make_request(
        uri=api_endpoint,
        headers=headers,
        method="POST",
        service="execute-api",
        data=json.dumps({"question": question}),
        region="us-west-2",
        access_key=credentials.access_key,
        secret_key=credentials.secret_key,
        security_token=credentials.token,
        data_binary=False
    )
    if response.status_code != 200:
        return "Request failed. Check the CloudWatch logs in your Lambda application."
    else:
        return response.json().get("answer")

***

###### 1
## <a>Part 1 - Extend the knowledge database that powers RAG</a>
([Go to top](#0))

LLMs are trained on massive datasets that allow them understand, summarize, and generate novel content. Most LLMs are trained on a wide range of public data so they can respond to many types of tasks or questions. But once trained, LLMs do not have the ability to access data beyond their training data. [Retrieval Augmented Generation (RAG)](https://aws.amazon.com/what-is/retrieval-augmented-generation/) extends the already powerful capabilities of LLMs to specific domains or an organization's internal knowledge base, all without the need to retrain the model. It is a cost-effective approach to improving LLM output so it remains relevant, accurate, and useful in various contexts.

Your GenAI application has already been deployed with RAG capabilities and specific instructions to only use the information from the stored documents to produce its answers. The knowledge base is stored in Amazon Kendra and contains a snapshot of the [AWS Lambda user guide](https://docs.aws.amazon.com/lambda/) as of October 2023.

### Amazon Kendra to store and search documents

When BuilderHub Create cloned the GenAI application, it created an Amazon Kendra index as part of it. [Amazon Kendra](https://aws.amazon.com/pm/kendra/) provides a solution to store and search documents using natural language processing. Your application uses Amazon Kendra to power document retrieval for RAG prompting. 

<div style="display: flex; align-items: center; justify-content: left; background-color:#330066; width:99%;"> 
        <img style="float: left; max-width: 100%; max-height:100%; margin: 15px;" src="images/MLU_robot.png" alt="MLU robot" width="100" height="100"/>
    <span style="color: white; padding-left: 10px; align: left; margin: 15px;">
        <b>Vector databases</b>
    <br/><br/>
        Amazon Kendra is not the only solution that can be used to power RAG-based generative AI application. 
        <br/><br/>Vector databases provide the ability to store and retrieve natural language data that has been encoded with an embedding model as high-dimensional points. 
        <br/><br/>Read <a style="color: lightskyblue;" href="https://aws.amazon.com/what-is/vector-databases/">here</a> to get more information about vector databases.
    </span>
</div>

The code below fetches the Kendra index that will be needed to retrieve related documents. We then define a function that, given a query, returns the list of stored documents that are relevant to it. 

In [3]:
from langchain.tools import tool

kendra = session.client("kendra", region_name="us-west-2")
response = kendra.list_indices()
KENDRA_INDEX_ID = response["IndexConfigurationSummaryItems"][0]["Id"]


@tool
def retrieve_context(query: str) -> dict:
    """Retrieve the list of documents from Kendra that are relevant to the query"""
    response = kendra.retrieve(
        IndexId=KENDRA_INDEX_ID,
        QueryText=query,
        PageNumber=1,
        PageSize=5,
    )
    documents = response["ResultItems"]
    document_ids = [document["DocumentId"] for document in documents]
    context = "\n".join([document["Content"] for document in documents])
    return {
        "question": query,
        "context": context,
        "document_ids": document_ids
    }


<div style="align: left; border: 4px solid lightcoral; text-align: left; margin: auto; padding-left: 20px; padding-right: 20px; width: 65%">
        <img style="float: left; margin: 15px;" src="images/MLU_question.png" alt="MLU question" width="100" height="100"/>
    <span style="padding: 20px; align: left;">
        <p><b>A note on Amazon Kendra</b><p/>
        <p>Please note that Amazon Kendra is not free of charge. If you decide to use it in your production projects, check out <a href="https://aws.amazon.com/kendra/pricing">its pricing</a>. Also note that the free tier option provides free usage of up to 750 hours for the first 30 days.</p>
        <br/>
    </span>
</div>

The Kendra document store, as it was created from the AWS Lambda user guide as of October 2023, misses knowledge about other AWS services. It also contains outdated information about AWS Lambda, as the service underwent updates between now and then. 

To demonstrate the shortcomings of RAG when the document store is incomplete or outdated, let's make requests to the live system with questions that are either not found in the database, or retrieve outdated or incomplete answers. 

In [4]:
unknown_questions = [
    "Is the .NET 8 runtime supported in AWS Lambda?",
    "What built-in algorithms are supported in SageMaker?",
]

display(Markdown("## Questions about AWS Services"))
for aws_q in unknown_questions:
    display(Markdown(f"**{aws_q}**"))
    display(Markdown(make_live_request(aws_q)))

## Questions about AWS Services

**Is the .NET 8 runtime supported in AWS Lambda?**

No, the .NET 8 runtime is not currently supported in AWS Lambda. The supported .NET runtimes for AWS Lambda are:

- .NET 6
- .NET 5
- .NET Core 3.1

The information provided in the context does not mention .NET 8 as a supported runtime for AWS Lambda. AWS Lambda typically adds support for new runtime versions over time, but .NET 8 has not been added yet.

**What built-in algorithms are supported in SageMaker?**

Request failed. Check the CloudWatch logs in your Lambda application.

Compare the responses above with current information found elsewhere: 

* On .NET8 in AwS Lambda: https://aws.amazon.com/blogs/compute/introducing-the-net-8-runtime-for-aws-lambda/
* On SageMaker built-in algorithms: https://docs.aws.amazon.com/sagemaker/latest/dg/algos.html

### Incomplete information in Amazon Kendra

Check that the retrieved documents from Kendra for the last question have low confidence, indicating that the relevant information to answer the question is missing from the document store:

In [5]:
question = unknown_questions[-1]
display(Markdown(f"Context for question: **{question}**"))
response = kendra.retrieve(
        IndexId=KENDRA_INDEX_ID,
        QueryText=question,
        PageNumber=1,
        PageSize=5,
    )
documents = response["ResultItems"]
documents

Context for question: **What built-in algorithms are supported in SageMaker?**

[{'Id': 'aef85dee-f588-4dbb-8fc5-0b737d4a7eff-9eb81109-6904-4c1a-9238-b2e5510e255f',
  'DocumentId': 's3://koachangmlucoursellmops-data-datasetbucket9be8b15b-j7cq8iolcazu/rag/lambda-developer-guide-231030/runtimes-avx2.md',
  'DocumentTitle': 'runtimes-avx2',
  'Content': '# Using AVX2 vectorization in Lambda<a name="runtimes-avx2"></a> Advanced Vector Extensions 2 \\(AVX2\\) is a vectorization extension to the Intel x86 instruction set that can perform single instruction multiple data \\(SIMD\\) instructions over vectors of 256 bits\\. For vectorizable algorithms with [highly parallelizable](https://en.wikipedia.org/wiki/Massively_parallel) operation, using AVX2 can enhance CPU performance, resulting in lower latencies and higher throughput\\. Use the AVX2 instruction set for compute\\-intensive workloads such as machine learning inferencing, multimedia processing, scientific simulations, and financial modeling applications\\. **Note** Lambda arm64 uses NEON SIMD architecture and does

### Add documents to the Amazon Kendra store

It seems apparent that including more relevant and comprehensive sources inside Kendra would improve the quality of the responses retrieved by the RAG system. Follow the set of instructions below to incorporate information into the system. 

<div style="align: left; border: 4px solid royalblue; text-align: left; margin: auto; padding-left: 20px; padding-right: 20px; width: 65%">
        <img style="float: left; max-width: 100%; max-height:100%; margin: 15px;" src="images/MLU_challenge.png" alt="MLU challenge" width=15% height=15%/>
    <span style="padding: 20px; align: left;">
        <p><b>Try it yourself!</b><p/>
        <p><b>Exercise 1.</b> You can augment the knowlegde accessible to the RAG service by incorporating new data sources into the document store.
        Follow these steps:</p>
            <ol>
                <li>From a Terminal in your Cloud Desktop, access your Data package and add data sources to extend the knowledge base of your RAG-based system.<br/>To do so, execute the cell below to get the exact commands that you need to run in the command line to create subfolders and place the provided files inside the Data package. Said files contain up-to-date information needed to correctly answer the questions about AWS services.</li><br/>
            </ol>
        <br/>
    </span>
</div>


In [6]:
display(Markdown("### Copy these commands and run them on your Cloud Desktop terminal to create subfolders and move files with additional information to your Data package."))

print(f"mkdir $(brazil-context workspace root)/src/{MAIN_PACKAGE_NAME}Data/public/rag/blogs")
print(f"mkdir $(brazil-context workspace root)/src/{MAIN_PACKAGE_NAME}Data/public/rag/sagemaker-developer-guide")

print(f"cp $(brazil-context workspace root)/src/{MAIN_PACKAGE_NAME}Experiment/notebooks/solutions/blogs/*.md $(brazil-context workspace root)/src/{MAIN_PACKAGE_NAME}Data/public/rag/blogs")
print(f"cp $(brazil-context workspace root)/src/{MAIN_PACKAGE_NAME}Experiment/notebooks/solutions/sagemaker-developer-guide/*.md $(brazil-context workspace root)/src/{MAIN_PACKAGE_NAME}Data/public/rag/sagemaker-developer-guide")

### Copy these commands and run them on your Cloud Desktop terminal to create subfolders and move files with additional information to your Data package.

mkdir $(brazil-context workspace root)/src/KoachangMLUCourseLLMOpsData/public/rag/blogs
mkdir $(brazil-context workspace root)/src/KoachangMLUCourseLLMOpsData/public/rag/sagemaker-developer-guide
cp $(brazil-context workspace root)/src/KoachangMLUCourseLLMOpsExperiment/notebooks/solutions/blogs/*.md $(brazil-context workspace root)/src/KoachangMLUCourseLLMOpsData/public/rag/blogs
cp $(brazil-context workspace root)/src/KoachangMLUCourseLLMOpsExperiment/notebooks/solutions/sagemaker-developer-guide/*.md $(brazil-context workspace root)/src/KoachangMLUCourseLLMOpsData/public/rag/sagemaker-developer-guide


<div style="align: left; border: 4px solid royalblue; text-align: left; margin: auto; padding-left: 20px; padding-right: 20px; width: 65%">
        <img style="float: left; max-width: 100%; max-height:100%; margin: 15px;" src="images/MLU_challenge.png" alt="MLU challenge" width=15% height=15%/>
    <span style="padding: 20px; align: left;">
        <br/><p>Continue with these steps:<p/>
        <ol start="2">
            <li>Edit file <code>{Alias}MLUCourseLLMOpsData/public/rag_version.txt</code> to <code>1.1</code> (or any version number different from the current version)</li><br/>
            <li>From a Terminal in your Cloud Desktop, build the Data application package. To do that, execute the cell below to get the exact commands that you need to run in the command line to trigger the build.<br/><br/>
            <b>This will output <code>Non-building package; no work to do</code> but that is correct.</b></li><br/>
        </ol>
        <br/>
    </span>
</div>


In [7]:
display(Markdown("### Copy these commands and run them on your Cloud Desktop terminal to build the Data package."))

print(f"cd $(brazil-context workspace root)/src/{MAIN_PACKAGE_NAME}Data")

print(f"brazil-build")

### Copy these commands and run them on your Cloud Desktop terminal to build the Data package.

cd $(brazil-context workspace root)/src/KoachangMLUCourseLLMOpsData
brazil-build


<div style="align: left; border: 4px solid royalblue; text-align: left; margin: auto; padding-left: 20px; padding-right: 20px; width: 65%">
        <img style="float: left; max-width: 100%; max-height:100%; margin: 15px;" src="images/MLU_challenge.png" alt="MLU challenge" width=15% height=15%/>
    <span style="padding: 20px; align: left;">
        <br/><p>Continue with these steps:<p/>
        <ol start="4">
            <li>From a Terminal in your Cloud Desktop, move to the CDK package, build it, and deploy the data stack. To do that, execute the cell below to get the exact commands that you need to run in the command line to trigger the build.<br/><br/>
            <b>This step will take around 5 minutes.</b></li><br/>
        </ol>
        <br/>
    </span>
</div>


In [8]:
display(Markdown("### Copy these commands and run them on your Cloud Desktop terminal to deploy the data stack via CDK."))

print(f"cd $(brazil-context workspace root)/src/{MAIN_PACKAGE_NAME}CDK")

print(f"brazil-build && brazil-build run cdk deploy {MAIN_PACKAGE_NAME}-Data-alpha")

### Copy these commands and run them on your Cloud Desktop terminal to deploy the data stack via CDK.

cd $(brazil-context workspace root)/src/KoachangMLUCourseLLMOpsCDK
brazil-build && brazil-build run cdk deploy KoachangMLUCourseLLMOps-Data-alpha


Once you have successfully deployed your data stack, run the cell below and check whether the answers returned by the live system are now correct:

In [14]:
display(Markdown("## Questions about AWS Services"))
for aws_q in unknown_questions:
    display(Markdown(f"**{aws_q}**"))
    display(Markdown(make_live_request(aws_q)))

## Questions about AWS Services

**Is the .NET 8 runtime supported in AWS Lambda?**

No, the .NET 8 runtime is not currently supported in AWS Lambda. The supported .NET runtimes for AWS Lambda are:

- .NET 6
- .NET 5
- .NET Core 3.1

The information provided in the context does not mention .NET 8 as a supported runtime for AWS Lambda. AWS Lambda typically adds support for new runtime versions over time, but .NET 8 has not been added yet.

**What built-in algorithms are supported in SageMaker?**

Request failed. Check the CloudWatch logs in your Lambda application.

***

###### 2
## <a>Part 2 - Add links to relevant documents in the API response</a>
([Go to top](#0))

In this next exercise you are going to add a helpful improvement to the service: incorporate relevant links to the documents that RAG used to produce the system answer. This feature can be useful to double check that the system is not hallucinating information, and enhances the user's trust on the system response. 

Your deployed GenAI application serves its response by means of code present in the `handler.py` file present in your main application package: `{Alias}MLUCourseLLMOps/src/{alias}_mlu_course_llm_ops/handler.py`. You will now make changes to this file to include the relevant links to the retrieved files in the body response of the Lambda application. Accordingly, you will also need to modify the package tests to account for the change in the function response.  

Follow the instructions below to apply the necessary modifications to the code, build the new version, and deploy it to your live system. 

<div style="align: left; border: 4px solid royalblue; text-align: left; margin: auto; padding-left: 20px; padding-right: 20px; width: 65%">
        <img style="float: left; max-width: 100%; max-height:100%; margin: 15px;" src="images/MLU_challenge.png" alt="MLU challenge" width=15% height=15%/>
    <span style="padding: 20px; align: left;">
        <p><b>Try it yourself!</b><p/>
        <p><b>Exercise 2.</b> You can add links to relevant documents by modifying the code in the main application package.
        Follow these steps in your VS Code IDE:</p>
            <ol>
                <li>Open the main package located in <code>{WORKSPACE_ROOT}/src/{Alias}MLUCourseLLMOps</code>.</li><br/>
                <li>Navigate to <code>{Alias}MLUCourseLLMOps/src/{alias}_mlu_course_llm_ops/handler.py</code>.</li><br/>
                <li>Incorporate the code shown below into the file <code>handler.py</code>. You need to make various edits:</li><br/>
                <ul>
                    <li>Import statements belong at the top.</li>
                    <li>Function <code>get_link_from_document_id</code> needs to be placed close to other utility functions in <code>handler.py</code>.</li>
                    <li>The <code>query_handler</code> function needs to be modified as shown below to add the relevant links.</li><br/>
                </ul>
                <li>Save the file <code>handler.py</code> when you're finished adding the modifications shown below.</li><br/>
            </ol>
        <br/>
    </span>
</div>



In [10]:
display(Markdown("### Click here to see the source code for the `handler.py` file"))

print(f"https://code.amazon.com/packages/{MAIN_PACKAGE_NAME}/blobs/mainline/--/src/{alias}_mlu_course_llm_ops/handler.py")

### Click here to see the source code for the `handler.py` file

https://code.amazon.com/packages/KoachangMLUCourseLLMOps/blobs/mainline/--/src/koachang_mlu_course_llm_ops/handler.py


### Modifications to be incorporated into the `handler.py` file

Run the cell below to display the code to be added to `handler.py` in the main application package. 

Click on "View as a scrollable element" to display the full contents of the file. Be careful when porting the code to the `handler.py` file. 

In [11]:
%pycat solutions/lab2_ex2_handler.txt


import re
from typing import Optional

...

@tool
def retrieve_context(query: str) -> dict:
    """Retrieve the list of documents from Kendra that are relevant to the query"""
    response = kendra.retrieve(
        IndexId=KENDRA_INDEX_ID,
        QueryText=query,
        PageNumber=1,
        PageSize=5,
    )
    documents = response["ResultItems"]
    document_ids = [document["DocumentId"] for document in documents]
    context = "\n".join([document["Content"] for document in documents])
    return {
        "question": query,
        "context": context,
        "document_ids": document_ids,
    }


# Function to return a link mapping the retrieved document to its URL
def get_link_from_document_id(document_id: str) -> Optional[str]:
    if match := re.search("s3://.*?/rag/lambda-developer-guide-231030/(.*).md$", document_id):
        path = match.group(1)
        return f"https://docs.aws.amazon.com/lambda/latest/dg/{path}.html"

    if match := re.search("s3://.*?/rag/sagemaker-d

<div style="align: left; border: 4px solid royalblue; text-align: left; margin: auto; padding-left: 20px; padding-right: 20px; width: 65%">
        <img style="float: left; max-width: 100%; max-height:100%; margin: 15px;" src="images/MLU_challenge.png" alt="MLU challenge" width=15% height=15%/>
    <span style="padding: 20px; align: left;">
    <br/>
    <p>Continue with these steps:</p>
            <ol start="5">
                <li>Locate the folder that contains the tests for the main application in <code>{WORKSPACE_ROOT}/src/{Alias}MLUCourseLLMOps/test/</code></li><br/>
                <li>Modify the tests located in file <code>test_basic_llm_service.py</code>.<br/>You need to modify three test functions:</li><br/>
                    <ul>
                        <li><code>test_lambda_handler_returning_correct_format</code> and</li>
                        <li><code>test_chain_receiving_contexts</code> and </li>
                        <li><code>test_guardrailed_output</code>.</li><br/>
                    </ul>
                <li>Save the file <code>test_basic_llm_service.py</code> when you're finished adding the modifications shown below.</li><br/>
            </ol>
        <br/>
    </span>
</div>



In [2]:
display(Markdown("### Click here to see the source code for the `test_basic_llm_service.py` file"))

print(f"https://code.amazon.com/packages/{MAIN_PACKAGE_NAME}/blobs/mainline/--/test/test_basic_llm_service.py")

### Click here to see the source code for the `test_basic_llm_service.py` file

https://code.amazon.com/packages/KoachangMLUCourseLLMOps/blobs/mainline/--/test/test_basic_llm_service.py


### Modifications to be incorporated into the `test_basic_llm_service.py` file

Run the cell below to display the code to be added to `test_basic_llm_service.py` in the main application package. 

Click on "View as a scrollable element" to display the full contents of the file. Be careful when porting the code to the `test_basic_llm_service.py` file. 

In [9]:
%pycat solutions/lab2_ex2_tests.txt

def test_lambda_handler_returning_correct_format(
        mock_retrieve_context, mock_get_chain, mock_event
    ):
    mock_event["body"] = '{"question": "fake question"}'
    mock_get_chain.return_value.invoke.return_value = "fake_answer"
    mock_retrieve_context.return_value = {
        "question": "fake question",
        "context": "Context Foo\nContext Bar",
        "document_ids": [
            "s3://fake-bucket/rag/lambda-developer-guide-231030/foo.md",
            "s3://fake-bucket/rag/blogs/bar.md"
            ]
        }

    response = lambda_handler(mock_event, None)

    assert response.get("statusCode") == HTTPStatus.OK
    assert json.loads(response.get("body")).get("answer") == "fake_answer"
    assert set(json.loads(response.get("body")).get("relevant_links")) == set([
        "https://docs.aws.amazon.com/lambda/latest/dg/foo.html",
        "https://aws.amazon.com/blogs/compute/bar/"
    ])
    

def test_chain_receiving_contexts(mock_aws, mock_prompt, mock_llm, mock_

<div style="align: left; border: 4px solid royalblue; text-align: left; margin: auto; padding-left: 20px; padding-right: 20px; width: 65%">
        <img style="float: left; max-width: 100%; max-height:100%; margin: 15px;" src="images/MLU_challenge.png" alt="MLU challenge" width=15% height=15%/>
    <span style="padding: 20px; align: left;">
    <br/>
    <p>Continue with these steps:</p>
            <ol start="8">
                <li>From a Terminal in your Cloud Desktop and while you are inside <code>{Alias}MLUCourseLLMOps</code>, build the main application package.</li><br/>
                <li>Move to the CDK package <code>{Alias}MLUCourseLLMOpsCDK</code>, build it and deploy the main service stack</li><br/>
            </ol>
            To do that, execute the cell below to get the exact commands that you need to run in the command line to trigger the builds and deployment.<br/>
        <br/>
    </span>
</div>

In [ ]:
display(Markdown("### Copy these commands and run them on your Cloud Desktop terminal to build the main application and the CDK packages and to deploy the main service stack."))

print(f"cd $(brazil-context workspace root)/src/{MAIN_PACKAGE_NAME}")
print(f"brazil-build release")

print(f"cd $(brazil-context workspace root)/src/{MAIN_PACKAGE_NAME}CDK")
print(f"brazil-build && brazil-build run cdk deploy {MAIN_PACKAGE_NAME}-Service-alpha -- --hotswap --exclusively")

<div style="display: flex; align-items: center; justify-content: left; background-color:#330066; width:99%;"> 
        <img style="float: left; max-width: 100%; max-height:100%; margin: 15px;" src="images/MLU_robot.png" alt="MLU robot" width="100" height="100"/>
    <span style="color: white; padding-left: 10px; align: left; margin: 15px;">
        <b>Did it succeed?</b>
    <br/><br/>
    If you see <code style="color: lightcoral;">BUILD SUCCEEDED</code> you have successfully implemented your changes!
    </span>
</div>

Once you have successfully deployed your main service stack, run the cells below and check whether the answers returned by the live system now contain the relevant links. 

In [2]:
# Modified function to make live requests and return the full json response rather than the "answer" key only
def make_live_request_links(question):
    credentials = session.get_credentials() 
    response = make_request(
        uri=api_endpoint,
        headers=headers,
        method="POST",
        service="execute-api",
        data=json.dumps({"question": question}),
        region="us-west-2",
        access_key=credentials.access_key,
        secret_key=credentials.secret_key,
        security_token=credentials.token,
        data_binary=False,
    )
    if response.status_code != 200:
        return "Request failed. Check the CloudWatch logs in your Lambda application."
    else:
        return response.json()

In [ ]:
unknown_questions = [
    "Is the .NET 8 runtime supported in AWS Lambda?",
    "What built-in algorithms are supported in SageMaker?",
]

display(Markdown("### Questions about AWS Services"))
for aws_q in unknown_questions:
    display(Markdown(f"**{aws_q}**"))
    response = make_live_request_links(aws_q)
    display(Markdown(response["answer"]))
    display(Markdown("_References_"))
    for link in response["relevant_links"]:
        display(Markdown(f" - {link}"))

### Checking that your modifications were successful

Do the responses above contain reference links? 

**Are the found references relevant to the questions at hand?**

***

###### 3
## <a>Part 3 - Commit your changes and update the pipeline</a>
([Go to top](#0))

As explained in Lab 1, there are several ways to trigger updates to the CloudFormation stacks that manage the deployment of the application. 

Deploying **using the CDK is temporary** and the deployment will eventually be overridden by a Pipelines deployment later, but can be useful for fast tests. This is so far the mechanism that we have used to update our service.

Deploying **via Pipelines is recommended for real workload**, as it commits lasting changes to the code repositories.

Once you're satisfied with the service improvements implemented in Exercises 1 and 2 above, it is time to push your changes into your packages and see them propagate to your pipeline. Follow the steps below to do so. 


<div style="align: left; border: 4px solid royalblue; text-align: left; margin: auto; padding-left: 20px; padding-right: 20px; width: 65%">
        <img style="float: left; max-width: 100%; max-height:100%; margin: 15px;" src="images/MLU_challenge.png" alt="MLU challenge" width=15% height=15%/>
    <span style="padding: 20px; align: left;">
        <p><b>Try it yourself!</b><p/>
        <p><b>Exercise 3.</b> It is time to make your changes permanent by committing them to your packages.
        Follow these steps:</p>
            <ol>
                <li>From a Terminal in your Cloud Desktop, navigate to the Data package. Then, using git, add, commit, and push your changes to the Data package.</li><br/>
            </ol>
            To do that, execute the cell below to get the exact commands that you need to run in the command line to commit the changes.<br/>
            <br/>
    </span>
</div>


In [4]:
display(Markdown("### Copy these commands and run them on your Cloud Desktop terminal to commit your changes to your Data package."))

print(f"cd $(brazil-context workspace root)/src/{MAIN_PACKAGE_NAME}Data")

print("git add -u\ngit add public/rag")
print('git commit -m "Add data sources to Kendra and bump data version"')
print("kinit -f\ngit push")

### Copy these commands and run them on your Cloud Desktop terminal to commit your changes to your Data package.

cd $(brazil-context workspace root)/src/KoachangMLUCourseLLMOpsData
git add -u
git add public/rag
git commit -m "Add data sources to Kendra and bump data version"
kinit -f
git push


<div style="align: left; border: 4px solid royalblue; text-align: left; margin: auto; padding-left: 20px; padding-right: 20px; width: 65%">
        <img style="float: left; max-width: 100%; max-height:100%; margin: 15px;" src="images/MLU_challenge.png" alt="MLU challenge" width=15% height=15%/>
    <span style="padding: 20px; align: left;">
        <p>Continue with these steps:<p/>
            <ol start="2">
                <li>From a Terminal in your Cloud Desktop, navigate to the main application package. Then, using git, add, commit, and push your changes to the main package.</li><br/>
                To do that, execute the cell below to get the exact commands that you need to run in the command line to commit the changes.<br/>
            </ol>
        <br/>
    </span>
</div>


In [1]:
display(Markdown("### Copy these commands and run them on your Cloud Desktop terminal to commit your changes to your main application package."))

print(f"cd $(brazil-context workspace root)/src/{MAIN_PACKAGE_NAME}")

print("git add -u")
print('git commit -m "Modify lambda handler to return relevant documents. Adapt tests."')
print("git push")

NameError: name 'Markdown' is not defined

Wait for the changes to propagate through the pipeline.

You can refresh and follow the status of your pipeline, which is located at:

In [ ]:
display(f"https://pipelines.amazon.com/pipelines/{MAIN_PACKAGE_NAME}")

<div style="align: left; border: 4px solid lightcoral; text-align: left; margin: auto; padding-left: 20px; padding-right: 20px; width: 65%">
        <img style="float: left; max-width: 100%; max-height:100%; margin: 15px;" src="images/MLU_question.png" alt="MLU solution" width=15% height=15%/>
    <span style="padding: 20px; align: left;">
        <p><br/><b>Challenge Help</b><p/>
        <p><b>Exercise 3.</b> While the changes are being propagated through the pipeline, you can click on the builds in progress and see how the different parts of the application get updated.</p>
        <p>Deployment via Pipelines might take up to one hour to complete.</p>
        <br/>
    </span>
</div>

<div style="display: flex; align-items: center; justify-content: left; background-color:#330066; width:99%;"> 
        <img style="float: left; max-width: 100%; max-height:100%; margin: 15px;" src="images/MLU_robot.png" alt="MLU robot" width="100" height="100"/>
    <span style="color: white; padding-left: 10px; align: left; margin: 15px;">
        <h3>Congratulations!</h3>
        You have completed Lab 2 of MLU's course Operationalizing Generative AI with LLMOps.
        <br/>
    </span>
</div>